# Detecting Credit Card Fraud

# INTRODUCTION (text)

# UNDERSTANDING AND EXPLAINING MACHINE LEARNING ALGORITHMS (text)

# UNDERSTANDING,CLEANING, and VISUALIZING THE DATA (text and code)

In [2]:
#importing packages
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np 
%matplotlib inline

In [38]:
# reading in files 
df = pd.read_csv('datasets/bs140513_032310.csv')

### Cleaning the Data

In [49]:
# method that removes quotations 
def remove_quotations(x):
    return x.strip("''")

# method that replaces age null value w/ -1 
def age_null(x):
    if x == 'U':
        return -1 
    return x 

# method that encodes gender into a number 
def gender_switch(x):
    if x == 'M':
        return 0
    if x == 'F':
        return 1
    if x == 'E':
        return 2
    else: 
        return -1

# method that cleans dataframe and returns cleaned version 
def clean_transaction(df):
    transaction_dataset = df.copy()
    transaction_dataset['customer'] = transaction_dataset['customer'].apply(remove_quotations) # remove quotation 
    transaction_dataset['age'] = transaction_dataset['age'].apply(remove_quotations).apply(age_null) # remove quotations and replaces null
    transaction_dataset['gender'] = transaction_dataset['gender'].apply(remove_quotations).apply(gender_switch) # remove quotation and encode gender
    transaction_dataset['zipcodeOri'] = transaction_dataset['zipcodeOri'].apply(remove_quotations).astype(int) # remove quotations 
    transaction_dataset['merchant'] = transaction_dataset['merchant'].apply(remove_quotations)
    transaction_dataset['zipMerchant'] = transaction_dataset['zipMerchant'].apply(remove_quotations).astype(int)
    transaction_dataset['category'] = transaction_dataset['category'].apply(remove_quotations)
    # transaction_dataset['category'] = transaction_dataset['category'].apply(encoder_categories)
    return transaction_dataset

In [50]:
clean_transaction = clean_transaction(df)

In [51]:
different_categories = clean_transaction['category'].unique().tolist()
category_dictionary = {} 
iterator = 0 
for i in different_categories:
    category_dictionary[i] = iterator
    iterator += 1  

# method that encodes the transaction categories into numbers 
def encoder_categories(x):
    return category_dictionary[x]

In [52]:
clean_transaction['category'] = clean_transaction['category'].apply(encoder_categories)

In [53]:
clean_transaction.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,C1093826151,4,0,28007,M348934600,28007,0,4.55,0
1,0,C352968107,2,0,28007,M348934600,28007,0,39.68,0
2,0,C2054744914,4,1,28007,M1823072687,28007,0,26.89,0
3,0,C1760612790,3,0,28007,M348934600,28007,0,17.25,0
4,0,C757503768,5,0,28007,M348934600,28007,0,35.72,0


Categories have now been encoded to numbers for easier manipulation. The following dictionary explains what each value means:

In [54]:
category_dictionary

{'es_transportation': 0,
 'es_health': 1,
 'es_otherservices': 2,
 'es_food': 3,
 'es_hotelservices': 4,
 'es_barsandrestaurants': 5,
 'es_tech': 6,
 'es_sportsandtoys': 7,
 'es_wellnessandbeauty': 8,
 'es_hyper': 9,
 'es_fashion': 10,
 'es_home': 11,
 'es_contents': 12,
 'es_travel': 13,
 'es_leisure': 14}

In [45]:
clean_transaction['zipcodeOri'].describe()

count    594643.0
mean      28007.0
std           0.0
min       28007.0
25%       28007.0
50%       28007.0
75%       28007.0
max       28007.0
Name: zipcodeOri, dtype: float64

In [46]:
clean_transaction['zipMerchant'].describe()

count    594643.0
mean      28007.0
std           0.0
min       28007.0
25%       28007.0
50%       28007.0
75%       28007.0
max       28007.0
Name: zipMerchant, dtype: float64

There are 10 columns in the dataset, but the two columns that specify zipcode only
contain one value and should be dropped.

In [47]:
clean_transaction = clean_transaction.drop(['zipcodeOri', 'zipMerchant'], axis=1)

There are 8 columns remaining. Step appears to be represent the time a transaction occurred since the first transaction was made. According to the paper published on researchgate.net, the Age column refers to a categorized age:

**0 --> <=18**

**1 --> 19-25**

**2 --> 26-35**

**3 --> 36-45**

**4 --> 46-55**

**5 --> 56-65**

**6 --> >65**

**U --> Unknown**

The Customer column is a unique ID assigned to each customer. There are customers in the dataset who have made multiple purchases, so this column may be useful to us. Gender represents the gender of a customer: 

**'M' -> Male**

**'F' -> Female**

**'E' -> Enterprise**

**'U' -> Unknown**

The Merchant column refers to a unique ID assigned to each merchant. There are merchants who have facilitated more than one transaction in the dataset, so this column may be useful to us.

Category and amount columns are self-evident, category represents the consumer category to which the purchased item belongs to, and amount refers to the numeric value of the purchase (likely in euros). 

The fraud column is categorized as follows: 0 is indicative of no fraud, 1 means the transaction was a fraud.

Additionally, there are no null values in the dataset, so no values seem to missing. This makes sense as the dataset we are dealing with is simulated.

### Understanding the Data

### Manipulating the Data (text and code)

### Visualizing the Data

# APPLYING MACHINE LEARNING (text and code) 

In [5]:
model = linear_model 
model.fit([TrainX, TrainY])
predictions = model.predict(Validate_or_test_X)

NameError: name 'linear_model' is not defined